In [5]:
import requests
import hexdump

In [6]:
data = requests.get(
    "https://quals.2018.volgactf.ru/files/9fbdd4dc5135f43ca537e9be33991ce5/capture.pcap",
    stream=True
).raw.read()

len(data)

1110016

Initially, opening the file in Wireshark greets us with

> The capture file appears to be damaged or corrupt.
>
> (pcap: File has 3579821647-byte packet, bigger than maximum of 262144)

Nevertheless, we can inspect the contents. We see two major protocols: SSH and MySQL. The MySQL data packets look interesting:

```
00000000  00 41 de ea 3c 11 64 87  88 5a f9 ba 08 00 45 00  |.A..<.d..Z....E.|
00000010  01 84 30 4f 40 00 3f 06  82 4c 2e a1 36 6f 5f d5  |..0O@.?..L..6o_.|
00000020  c2 f3 0c ea d8 6e 16 6b  c0 a2 19 eb 26 d6 80 18  |.....n.k....&...|
00000030  01 d6 3f 16 00 00 01 01  08 0a 00 0e 08 19 00 0b  |..?.............|
00000040  dc 61 42 00 00 cb 00 23  05 a9 5a 22 02 00 00 00  |.aB....#..Z"....|
00000050  41 00 00 00 88 9c 00 00  00 00 00 00 00 00 00 00  |A...............|
00000060  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000070  00 00 00 02 80 00 00 00  00 00 00 00 81 00 00 00  |................|
00000080  00 00 00 00 ff 5e 8f 93  4c 00 00 cc 00 23 05 a9  |.....^..L....#..|
00000090  5a 02 02 00 00 00 4b 00  00 00 d3 9c 00 00 08 00  |Z.....K.........|
000000a0  08 00 00 00 00 00 00 00  07 00 00 1a 00 00 00 00  |................|
000000b0  00 00 01 20 00 a0 55 00  00 00 00 06 03 73 74 64  |... ..U......std|
000000c0  04 21 00 21 00 08 00 6d  61 67 69 63 64 62 00 42  |.!.!...magicdb.B|
000000d0  45 47 49 4e 07 c7 d8 22  3a 00 00 cd 00 23 05 a9  |EGIN...":....#..|
000000e0  5a 13 02 00 00 00 39 00  00 00 0c 9d 00 00 00 00  |Z.....9.........|
000000f0  6d 00 00 00 00 00 01 00  07 6d 61 67 69 63 64 62  |m........magicdb|
00000100  00 05 75 73 65 72 73 00  03 03 0f 0f 04 ff 00 ff  |..users.........|
00000110  00 07 b0 2d b2 75 54 00  00 ce 00 23 05 a9 5a 1e  |...-.uT....#..Z.|
00000120  02 00 00 00 53 00 00 00  5f 9d 00 00 00 00 6d 00  |....S..._.....m.|
00000130  00 00 00 00 01 00 02 00  03 ff f8 80 00 00 00 10  |................|
00000140  56 6f 6c 75 70 74 75 6f  75 73 45 78 70 65 72 74  |VoluptuousExpert|
00000150  19 32 6e 72 66 70 37 68  75 6c 37 35 67 32 6d 74  |.2nrfp7hul75g2mt|
00000160  67 67 32 33 70 30 70 73  6d 67 ba 67 5b a2 20 00  |gg23p0psmg.g[. .|
00000170  00 cf 00 23 05 a9 5a 10  02 00 00 00 1f 00 00 00  |...#..Z.........|
00000180  7e 9d 00 00 00 00 a6 00  00 00 00 00 00 00 bd 4c  |~..............L|
00000190  e2 f7                                             |..|
00000192
```

A username and a password? _interesting_!

By comparing several samples:
```
00000130  00 00 00 00 01 00 02 00  03 ff f8 80 00 00 00 10  |................|
00000140  56 6f 6c 75 70 74 75 6f  75 73 45 78 70 65 72 74  |VoluptuousExpert|
00000150  19 32 6e 72 66 70 37 68  75 6c 37 35 67 32 6d 74  |.2nrfp7hul75g2mt|
00000160  67 67 32 33 70 30 70 73  6d 67 ba 67 5b a2 20 00  |gg23p0psmg.g[. .|

00000130  00 00 00 00 01 00 02 00  03 ff f8 83 00 00 00 0a  |................|
00000140  48 75 72 74 42 65 62 65  36 39 1a 72 76 6b 34 36  |HurtBebe69.rvk46|
00000150  76 61 61 73 66 71 73 66  6d 6b 61 76 69 70 34 31  |vaasfqsfmkavip41|
00000160  69 6a 39 35 62 aa a4 a2  13 20 00 00 de 00 23 05  |ij95b.... ....#.|

000002c0  00 01 00 02 00 03 ff f8  84 00 00 00 0c 55 6e 6c  |.............Unl|
000002d0  75 63 6b 79 53 61 74 61  6e 1a 6a 72 64 74 6d 65  |uckySatan.jrdtme|
000002e0  30 76 63 31 62 70 61 30  62 6a 71 68 76 69 31 75  |0vc1bpa0bjqhvi1u|
000002f0  30 63 6e 6e 5b ad f1 00  20 00 00 e3 00 23 05 a9  |0cnn[... ....#..|

00000450  01 00 02 00 03 ff f8 81  00 00 00 11 50 61 74 68  |............Path|
00000460  65 74 69 63 5f 42 65 62  65 32 30 30 30 1a 36 76  |etic_Bebe2000.6v|
00000470  33 6e 39 74 71 64 63 36  31 64 68 32 6c 66 73 35  |3n9tqdc61dh2lfs5|
00000480  34 6a 35 66 62 74 34 35  d3 41 b4 a6 20 00 00 d4  |4j5fbt45.A.. ...|
```

The formatting seems to be something like:
```
prefix:          03 ff f8 ?? ?? ?? ??
username length: 10
username:        56 6f 6c 75 70 74 75 6f 75 73 45 78 70 65 72 74
password length: 19
password:        32 6e 72 66 70 37 68 75 6c 37 35 67 32 6d 74 67 67 32 33 70 30 70 73 6d 67
```

So, to find all usernames/passwords, we search for the next occurrence of `03 ff f8`, then step forwards a few bytes, read the username length, read the username, read the password length, read the password. Rinse and repeat.

In [7]:
class Reader:
    def __init__(self, data):
        self.data = data
        self.i = 0
        
    def find(self, content):
        idx = self.data[self.i:].find(content)
        if idx == -1:
            return False
        self.i += idx
        return True
        
    def read(self, length):
        d = self.data[self.i:self.i+length]
        self.i += length
        return d

In [23]:
entries = {}
        
r = Reader(data)    

while True:
    if not r.find(b"\x03\xff\xf8"):
        break
    r.read(4)
    r.read(3)
    
    user_length = int.from_bytes(r.read(1), "big")
    username = r.read(user_length)
    
    pass_length = int.from_bytes(r.read(1), "big")
    password = r.read(pass_length)
    
    entries[username] = password
    
len(entries)

498

498 entries is not too bad, so let's just try to loop over all of them.

In [ ]:
s = requests.Session()

for username, password in entries.items():
    result = s.post("http://master.quals.2018.volgactf.ru:3333/", data={'uname': username, 'psw': password}).text
    if result != "No flag for you!":
        print(username, password, result)

b'admin' b'aep7Woo9eef7quiedooPh0oow' VolgaCTF{PLA1N_TEXT_REPLICATION_IS_@_B@D_THING}
